In [210]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import random
import tqdm
import warnings
warnings.simplefilter('ignore')

In [137]:
past = pd.read_csv('PastLoans.csv')
new = pd.read_csv('NewApplications_1_Round1.csv')
past.set_index(['id'], inplace=True)
new.set_index(['id'], inplace=True)
proba = pd.read_csv('regression_proba.csv')


In [138]:
proba

,prediction
0,0.137375
1,0.119819
2,0.114911
3,0.133836
4,0.121650
...,...
99995,0.166676
99996,0.457915
99997,0.336487
99998,0.159271


In [139]:
new.describe()

,married,income,digital1
count,100000.000000,100000.000000,100000.000000
mean,0.620580,38368.327340,0.376222
std,0.485245,74877.380095,0.212271
min,0.000000,0.000000,0.000000
25%,0.000000,17661.000000,0.201000
50%,1.000000,24638.000000,0.376000
75%,1.000000,36932.250000,0.535000
max,1.000000,1000000.000000,1.000000


In [140]:
new_2 = pd.read_csv('NewApplications_1_Round2.csv')
new_2.set_index(['id'], inplace=True)

In [141]:
new_2.describe()

,married,income,digital1
count,100000.000000,100000.000000,100000.000000
mean,0.624380,37730.241440,0.375153
std,0.484285,71875.534362,0.212992
min,0.000000,0.000000,0.000000
25%,0.000000,17573.000000,0.198000
50%,1.000000,24491.000000,0.375000
75%,1.000000,36924.250000,0.534000
max,1.000000,1000000.000000,1.000000


In [142]:
results = pd.read_csv('profit_12.csv')
results.set_index(['id'], inplace=True)

In [143]:
results.describe()


,own,competing2,competing3,borrowertype,accepted,default,profit
count,100000.000000,78702.000000,91537.000000,100000.000000,100000.000000,100000.000000,36774.000000
mean,0.265085,0.310478,0.289839,1.999990,0.367740,0.179620,-200.788010
std,0.146896,0.214621,0.214404,0.816503,0.482193,0.383873,5291.462132
min,0.088073,0.027000,0.007000,1.000000,0.000000,0.000000,-10000.000000
25%,0.170742,0.164427,0.131071,1.000000,0.000000,0.000000,1235.438000
50%,0.211428,0.206340,0.220819,2.000000,0.000000,0.000000,1811.931500
75%,0.320898,0.373832,0.393944,3.000000,1.000000,0.000000,2405.251500
max,1.000000,0.999697,1.000000,3.000000,1.000000,1.000000,10000.000000


In [144]:
results.head()

,own,competing2,competing3,borrowertype,accepted,default,profit
id,,,,,,,
100001,0.178253,NaN,0.174741,3,0,0,NaN
100002,0.155131,0.469120,0.093990,1,0,0,NaN
100003,0.148830,0.099437,0.099718,3,0,0,NaN
100004,0.173515,0.418875,0.218949,3,1,0,1735.152
100005,0.157499,0.514527,0.145152,1,1,0,1574.986


In [360]:
def simulation(df):
    results = df

    # generate random borrower type for simulation
    x = np.concatenate((np.ones((33333,)), 2*np.ones((33333,)), 3*np.ones((33334,))))
    np.random.shuffle(x)
    #df['borrowertype'] = x

    df['interestrate_borrower1'] = df['own']
    df['interestrate_borrower2'] = df['competing2']
    df['interestrate_borrower3'] =df['competing3']

    # measure the accepted loan based on the borrowertype
    sub_df1 = df[df['borrowertype'].isin([1])]
    sub_df1['interestrate_borrower1'] = sub_df1['own'] - 0.02
    df[df['borrowertype'].isin([1])]= sub_df1

    sub_df2 = df[df['borrowertype'].isin([2])]
    sub_df2['interestrate_borrower2'] = sub_df2['competing2'] - 0.02
    df[df['borrowertype'].isin([2])] = sub_df2

    sub_df3 = df[df['borrowertype'].isin([3])]
    sub_df3['interestrate_borrower3'] = sub_df3['competing3'] - 0.02
    df[df['borrowertype'].isin([3])] = sub_df3

    df['winner'] = np.nanargmin(np.array(df[['interestrate_borrower1', 'interestrate_borrower2', 'interestrate_borrower3']]), axis=1)+1

    # measure profit for each borrower type 
    sub_df1 = df[df['winner'].isin([1])][~df['own'].isna()]
    sub_df1['profit1'] = 10000*(sub_df1['own']*(1-sub_df1['default']))-sub_df1['default']*10000

    sub_df2 = df[df['winner'].isin([2])][~df['competing2'].isna()]
    sub_df2['profit2'] = 10000*(sub_df2['competing2']*(1-sub_df2['default']))-sub_df2['default']*10000

    sub_df3 = df[df['winner'].isin([3])][~df['competing3'].isna()]
    sub_df3['profit3'] = 10000*(sub_df3['competing3']*(1-sub_df3['default']))-sub_df3['default']*10000

    profits = np.array([sub_df1['profit1'].sum(), sub_df2['profit2'].sum(), sub_df3['profit3'].sum()])
    market_share = np.array([sub_df1.shape[0], sub_df2.shape[0], sub_df3.shape[0]])

    return df, profits, market_share

In [361]:
results = pd.read_csv('profit_12.csv')
results.set_index(['id'], inplace=True)

df, profit, market_share = simulation(results)
profit

array([ -7383778.292, -12584258.854,  -4275397.532])

In [359]:
df.isna().sum()

own                           0
competing2                21298
competing3                 8463
borrowertype                  0
accepted                      0
default                       0
profit                    63226
interestrate_borrower1        0
interestrate_borrower2    21298
interestrate_borrower3     8463
winner                        0
dtype: int64

### Tests to do
- put higher margin for the 2/3 in the middle (after deleting the defaulters), 2/3 randomly as before, 
- set the low margin to 1.9 and then tune the hyperparameter between 3.2 and 3.9

In [362]:
proba.head()

,prediction,own,Break_even_rates,selection,interest_rates
0,0.137375,0.137375,0.159253,1.0,2.059253
1,0.119819,0.119819,0.136131,1.0,2.036131
2,0.114911,0.114911,0.129830,1.0,2.029830
3,0.133836,0.133836,0.154515,1.0,2.054515
4,0.121650,0.121650,0.138499,1.0,2.038499


In [364]:
def test_strategy_middle(df, proba, m1, m2, p_low, p_mid, p_high, nb_simulations):
    
    proba['own'] = proba['prediction'].values
    proba['Break_even_rates'] = proba['own']/(1-proba['own'])

    # distribute the margins categories 
    proba['selection'] = np.ones((df.shape[0]))
    proba[proba['own']<p_mid][proba['own']>p_low]['selection'] = 2
    proba[proba['own']>p_high]['selection'] = 0

    # this is where we will do the hyperparameter tuning
    proba.loc[proba['selection']==0,'interest_rates'] = np.nan
    proba.loc[proba['selection']==1,'interest_rates'] = proba['Break_even_rates']+m1
    proba.loc[proba['selection']==2,'interest_rates'] = proba['Break_even_rates']+m2

    df['own'] = proba['interest_rates'].values

    profits = pd.DataFrame(np.zeros((nb_simulations,3)))
    for n in range(nb_simulations):
        random.seed(n)
        _, profit, _ = simulation(df)
        profits.iat[n,0] = profit[0]
        profits.iat[n,1] = profit[1]
        profits.iat[n,2] = profit[2]

    return profits

In [365]:
sim_results = pd.DataFrame({'Low Margin': [], 'High Margin': [], 'High percentile': [], 'Low percentile': [], 'Profit 1': [], 'Profit 2': [], 'Profit 3': []})
sim_results.append(pd.DataFrame([np.concatenate((np.array([0,0,0,0]),np.array([0,0,0])))], columns=sim_results.columns))

,Low Margin,High Margin,High percentile,Low percentile,Profit 1,Profit 2,Profit 3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [366]:
sim_results = pd.DataFrame({'Low Margin': [], 'High Margin': [], 'High percentile': [], 'Mid percentile': [], 'Low percentile': [], 'Profit 1': [], 'Profit 2': [], 'Profit 3': []})

print('For proba < p_low, break-even rate + m1')
print('For p_low < proba < p_mid, break-even rate + m2')
print('For p_mid < proba < p_high, break-even rate + m1')
print('For proba > p_out, no interest rate')

i=0
sim_outputs = np.zeros((8,))
m1=1.9
from tqdm import tqdm
for m2 in tqdm([3.2, 3.5, 3.8]):
    for perc_high in tqdm(np.linspace(70, 85, 5)):
        for perc_mid in tqdm(np.linspace(30, 70, 10)):
            for perc_low in tqdm(np.linspace(0, 30, 10)):
                p_low = np.percentile(proba['prediction'], perc_low)
                p_mid = np.percentile(proba['prediction'], perc_mid)
                p_high = np.percentile(proba['prediction'], perc_high)
                results = pd.read_csv('profit_12.csv')
                results.set_index(['id'], inplace=True)
                profits = test_strategy_middle(results, proba, m1, m2, p_low, p_mid, p_high, nb_simulations=5)
                sim_output = np.concatenate((np.array([m1,m2,p_low, p_mid, p_high,]),np.array(profits.mean().values)))
                sim_outputs = np.vstack((sim_outputs, sim_output))


For proba < p_low, break-even rate + m1
For p_low < proba < p_mid, break-even rate + m2
For p_mid < proba < p_high, break-even rate + m1
For proba > p_out, no interest rate


  0%|          | 0/3 [00:00<?, ?it/s]






















100%|██████████| 10/10 [00:09<00:00,  1.06it/s]























100%|██████████| 10/10 [00:07<00:00,  1.34it/s]























100%|██████████| 10/10 [00:08<00:00,  1.16it/s]























100%|██████████| 10/10 [00:09<00:00,  1.05it/s]

















In [323]:
sim_results = pd.DataFrame(sim_outputs)

sim_results.columns = ['Low Margin', 'High Margin', 'High percentile', 'Low percentile', 'Profit 1', 'Profit 2', 'Profit 3']

In [325]:
# hyperparameters: high margin, low percentile, mid percentile, high percentile (percentiles based on default probabilities)
# distribution of interest rates:
# bellow low percentile (less probability to default): interest rate = break-even + low margin --> to maximise market share
# between low percentile, mid percentile: interest rate = break-even + high margin --> to maximise return
# between mis percentile, high percentile: interest rate = break-even + low margin --> to minimise risk
# above high percentile: no interest rate --> to avoid risk 
sim_results.describe()

,Low Margin,High Margin,High percentile,Low percentile,Profit 1,Profit 2,Profit 3
count,1401.000000,1401.000000,1401.000000,1401.000000,1.401000e+03,1.401000e+03,1.401000e+03
mean,1.898644,3.497502,0.240028,0.108070,3.469015e+07,-1.693459e+06,1.285330e+07
std,0.050762,0.220780,0.012630,0.011816,9.274643e+05,8.200486e+04,3.503875e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-1.925173e+06,0.000000e+00
25%,1.900000,3.300000,0.231258,0.096782,3.471493e+07,-1.738814e+06,1.281642e+07
50%,1.900000,3.500000,0.237192,0.109959,3.471493e+07,-1.691600e+06,1.285895e+07
75%,1.900000,3.700000,0.246296,0.117116,3.471493e+07,-1.648233e+06,1.290683e+07
max,1.900000,3.800000,0.263117,0.124609,3.471493e+07,0.000000e+00,1.309112e+07


In [326]:
sim_results

,Low Margin,High Margin,High percentile,Low percentile,Profit 1,Profit 2,Profit 3
0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
1,1.9,3.2,0.225984,0.087226,3.471493e+07,-1.724601e+06,1.288984e+07
2,1.9,3.2,0.225984,0.089811,3.471493e+07,-1.799686e+06,1.296625e+07
3,1.9,3.2,0.225984,0.092201,3.471493e+07,-1.724890e+06,1.289465e+07
4,1.9,3.2,0.225984,0.094672,3.471493e+07,-1.737152e+06,1.290608e+07
...,...,...,...,...,...,...,...
1396,1.9,3.8,0.263117,0.118459,3.471493e+07,-1.703655e+06,1.287407e+07
1397,1.9,3.8,0.263117,0.120084,3.471493e+07,-1.693489e+06,1.285757e+07
1398,1.9,3.8,0.263117,0.121738,3.471493e+07,-1.773345e+06,1.294146e+07
1399,1.9,3.8,0.263117,0.123226,3.471493e+07,-1.622107e+06,1.279102e+07


In [139]:
new_predictions = new_2
# add the predictions as a new column !!!

In [ ]:
new_predictions['prediction'].quantile(np.linspace(1, 0, 10))

In [ ]:
new_predictions['Break_even_rates'] = new_predictions ['prediction']/(1-new_predictions ['prediction'])
new_predictions['payoff_Break_even_rates'] = -10000 + 10000*(1-new_predictions ['prediction'])*(1+ new_predictions['Break_even_rates'])

# here take 1/3 randomly: this is where we input our targeted applicants based on percentile and quantile
nums = np.ones(100000)
nums[:33333] = 0
np.random.shuffle(nums)
new_predictions['random_selection'] =np.array(nums)

new_predictions['premium']= new_predictions['Break_even_rates'] + 0.019
new_predictions['payoff_premium'] = -10000 + 10000*(1-new_predictions ['prediction'])*(1+ new_predictions['premium'])

# this is where we will do the hyperparameter tuning
new_predictions.loc[new_predictions['random_selection'] == 0,'interest_rates'] = new_predictions['Break_even_rates'] + 0.019
new_predictions.loc[new_predictions['random_selection'] == 1,'interest_rates'] = new_predictions['Break_even_rates'] + 0.035
new_predictions['profit_model'] = -10000 + 10000*(1-new_predictions ['prediction'])*(1+ new_predictions['interest_rates'])
model = new_predictions[['random_selection','prediction','Break_even_rates','interest_rates','profit_model']]